# Global Nearshore Slopes 

Notebook environment to migrate csv files to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Import standard packages
import os
import pathlib
import sys

import numpy as np
#import geopandas as gpd
 
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import glob
import itertools
import json
import copy
from itertools import chain
from shapely import wkb

# Import custom functionality
# sys.path.append('c:/Windows/System32/coclicodata/src')#CLENMAR ADDED

from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\02_Exposure\Slopes-DOC"
#gca_data_dir = p_drive.joinpath("11209197-018-global-coastal-atlas","MSc_students","ClenmarRowe")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"AppData\Local\miniconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml" ###Changed from anaconda to miniconda  - added the new package udunits file
    )
)


<IPython.core.display.Javascript object>

In [3]:
# Project paths & files (manual input)
dataset_dir = pathlib.Path().joinpath(gca_data_dir,"Nearshore_Slopes")
dataset_dir_path = dataset_dir.joinpath("Nearshore_Slopes_original.nc")
CF_dir = dataset_dir.joinpath("CF")  # directory to save output CF check files
CF_dir

WindowsPath('P:/11209197-018-global-coastal-atlas/MSc_students/ClenmarRowe/Data/All_Datasets/Orig_Datasets/02_Exposure/Slopes-DOC/Nearshore_Slopes/CF')

<IPython.core.display.Javascript object>

In [4]:
filename=dataset_dir.joinpath("nearshore_slopes.csv")


df = pd.read_csv(filename, index_col=None, header=0)
# columns_to_remove = range(4, len(df.columns)-1)
# df = df.drop(df.columns[columns_to_remove], axis=1)
df

,X,Y,dc,slope,error_code
0,-26.3728,-58.4052,20.485961,0.011182,6
1,-26.3890,-58.4079,20.485961,0.011312,6
2,-26.4057,-58.4098,20.485961,0.010914,6
3,-26.4227,-58.4112,20.485961,0.010500,6
4,-26.4396,-58.4123,20.485961,3.269054,0
...,...,...,...,...,...
783433,48.1141,67.6111,8.504953,NaN,4
783434,48.1203,67.6145,8.504953,0.001116,2
783435,47.8598,67.5380,8.508141,0.001609,2
783436,47.8671,67.5357,8.509773,NaN,4


<IPython.core.display.Javascript object>

In [5]:
ds = xr.Dataset.from_dataframe(df)
ds

<xarray.Dataset>
Dimensions:     (index: 783438)
Coordinates:
  * index       (index) int64 0 1 2 3 4 5 ... 783433 783434 783435 783436 783437
Data variables:
    X           (index) float64 -26.37 -26.39 -26.41 -26.42 ... 47.86 47.87 53.9
    Y           (index) float64 -58.41 -58.41 -58.41 ... 67.54 67.54 68.94
    dc          (index) float64 20.49 20.49 20.49 20.49 ... 8.508 8.51 11.54
    slope       (index) float64 0.01118 0.01131 0.01091 ... 0.001609 nan nan
    error_code  (index) int64 6 6 6 6 0 0 0 0 3 6 6 0 ... 8 6 0 0 0 4 4 2 2 4 4

<IPython.core.display.Javascript object>

In [6]:
# Write the xarray dataset to a netCDF file
import os
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

ds.to_netcdf(str(dataset_dir_path))

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [7]:
# open datasets (only first file, rest is the same)
ds = xr.open_dataset(dataset_dir_path)

# check original dataset
ds

<xarray.Dataset>
Dimensions:     (index: 783438)
Coordinates:
  * index       (index) int64 0 1 2 3 4 5 ... 783433 783434 783435 783436 783437
Data variables:
    X           (index) float64 ...
    Y           (index) float64 ...
    dc          (index) float64 ...
    slope       (index) float64 ...
    error_code  (index) int64 ...

<IPython.core.display.Javascript object>

In [8]:
%%capture cap --no-stderr
# check original CF compliancy (for first file)

check_compliancy(testfile=dataset_dir_path, 
                 working_dir=CF_dir
                 )

<IPython.core.display.Javascript object>

In [9]:
# save original CF compliancy (for first file)
save_compliancy(cap, testfile=dataset_dir_path, working_dir=CF_dir)

{'P:\\11209197-018-global-coastal-atlas\\MSc_students\\ClenmarRowe\\Data\\All_Datasets\\Orig_Datasets\\02_Exposure\\Slopes-DOC\\Nearshore_Slopes\\Nearshore_Slopes_original.nc': {'warnings': '8', 'errors': '0'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [10]:
# open datasets (only first file, rest is the same)
ds = xr.open_dataset(dataset_dir_path)

ds

<xarray.Dataset>
Dimensions:     (index: 783438)
Coordinates:
  * index       (index) int64 0 1 2 3 4 5 ... 783433 783434 783435 783436 783437
Data variables:
    X           (index) float64 ...
    Y           (index) float64 ...
    dc          (index) float64 ...
    slope       (index) float64 ...
    error_code  (index) int64 ...

<IPython.core.display.Javascript object>

In [11]:
import json

# NetCDF attribute alterations by means of metadata template
metadata_json=dataset_dir.joinpath("metadata_Nearshore_slope.json")

f_global = open(metadata_json)
meta_global = json.load(f_global)


for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    ds.attrs[attr_name] = attr_val

ds.attrs['Conventions'] = "CF-1.8"
ds

<xarray.Dataset>
Dimensions:     (index: 783438)
Coordinates:
  * index       (index) int64 0 1 2 3 4 5 ... 783433 783434 783435 783436 783437
Data variables:
    X           (index) float64 ...
    Y           (index) float64 ...
    dc          (index) float64 ...
    slope       (index) float64 ...
    error_code  (index) int64 ...
Attributes: (12/21)
    TITLE:               Global Distribution of Nearshore Slopes
    TITLE_ABBREVIATION:  gsns
    DESCRIPTION:         Global nearshore slope estimations using an interpol...
    SHORT_DESCRIPTION:   Global nearshore slope estimations
    INSTITUTION:         Deltares/TU Twente
    PROVIDERS:           {"name": "Deltares/TU Twente", "url": "http://www.de...
    ...                  ...
    DOI:                 https://doi.org/10.5194/essd-11-515-2019
    LONG_NAME:           GLOBAL_DISTRIBUTION_OF_NEARSHORE_SLOPES
    UNITS:               Decimal degrees for Latitude and Longitude, Positive...
    COMMENT:             Error or Warning code from the calculation: 0: No er...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [12]:
ds = ds.rename_dims({"index": "nstations"})

ds

<xarray.Dataset>
Dimensions:     (nstations: 783438)
Coordinates:
  * index       (nstations) int64 0 1 2 3 4 ... 783434 783435 783436 783437
Dimensions without coordinates: nstations
Data variables:
    X           (nstations) float64 ...
    Y           (nstations) float64 ...
    dc          (nstations) float64 ...
    slope       (nstations) float64 ...
    error_code  (nstations) int64 ...
Attributes: (12/21)
    TITLE:               Global Distribution of Nearshore Slopes
    TITLE_ABBREVIATION:  gsns
    DESCRIPTION:         Global nearshore slope estimations using an interpol...
    SHORT_DESCRIPTION:   Global nearshore slope estimations
    INSTITUTION:         Deltares/TU Twente
    PROVIDERS:           {"name": "Deltares/TU Twente", "url": "http://www.de...
    ...                  ...
    DOI:                 https://doi.org/10.5194/essd-11-515-2019
    LONG_NAME:           GLOBAL_DISTRIBUTION_OF_NEARSHORE_SLOPES
    UNITS:               Decimal degrees for Latitude and Longitude, Positive...
    COMMENT:             Error or Warning code from the calculation: 0: No er...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [13]:

# rename variables, if necessary
ds = ds.rename_vars(
       {"X": "lon", "Y": "lat"
         #,"index":"nstations" }
       }
  )
ds

<xarray.Dataset>
Dimensions:     (nstations: 783438)
Coordinates:
  * index       (nstations) int64 0 1 2 3 4 ... 783434 783435 783436 783437
Dimensions without coordinates: nstations
Data variables:
    lon         (nstations) float64 ...
    lat         (nstations) float64 ...
    dc          (nstations) float64 ...
    slope       (nstations) float64 ...
    error_code  (nstations) int64 ...
Attributes: (12/21)
    TITLE:               Global Distribution of Nearshore Slopes
    TITLE_ABBREVIATION:  gsns
    DESCRIPTION:         Global nearshore slope estimations using an interpol...
    SHORT_DESCRIPTION:   Global nearshore slope estimations
    INSTITUTION:         Deltares/TU Twente
    PROVIDERS:           {"name": "Deltares/TU Twente", "url": "http://www.de...
    ...                  ...
    DOI:                 https://doi.org/10.5194/essd-11-515-2019
    LONG_NAME:           GLOBAL_DISTRIBUTION_OF_NEARSHORE_SLOPES
    UNITS:               Decimal degrees for Latitude and Longitude, Positive...
    COMMENT:             Error or Warning code from the calculation: 0: No er...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [14]:

# set some data variables to coordinates to avoid duplication of dimensions in later stage
ds = ds.set_coords(["lon", "lat"])

ds

<xarray.Dataset>
Dimensions:     (nstations: 783438)
Coordinates:
  * index       (nstations) int64 0 1 2 3 4 ... 783434 783435 783436 783437
    lon         (nstations) float64 ...
    lat         (nstations) float64 ...
Dimensions without coordinates: nstations
Data variables:
    dc          (nstations) float64 ...
    slope       (nstations) float64 ...
    error_code  (nstations) int64 ...
Attributes: (12/21)
    TITLE:               Global Distribution of Nearshore Slopes
    TITLE_ABBREVIATION:  gsns
    DESCRIPTION:         Global nearshore slope estimations using an interpol...
    SHORT_DESCRIPTION:   Global nearshore slope estimations
    INSTITUTION:         Deltares/TU Twente
    PROVIDERS:           {"name": "Deltares/TU Twente", "url": "http://www.de...
    ...                  ...
    DOI:                 https://doi.org/10.5194/essd-11-515-2019
    LONG_NAME:           GLOBAL_DISTRIBUTION_OF_NEARSHORE_SLOPES
    UNITS:               Decimal degrees for Latitude and Longitude, Positive...
    COMMENT:             Error or Warning code from the calculation: 0: No er...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [15]:
#Drop index from coordinates list
# ds=ds.rename_dims({"index": "nstations"})
ds=ds.drop_vars("index")
ds = ds.set_coords(["error_code"])

ds

<xarray.Dataset>
Dimensions:     (nstations: 783438)
Coordinates:
    lon         (nstations) float64 ...
    lat         (nstations) float64 ...
    error_code  (nstations) int64 ...
Dimensions without coordinates: nstations
Data variables:
    dc          (nstations) float64 ...
    slope       (nstations) float64 ...
Attributes: (12/21)
    TITLE:               Global Distribution of Nearshore Slopes
    TITLE_ABBREVIATION:  gsns
    DESCRIPTION:         Global nearshore slope estimations using an interpol...
    SHORT_DESCRIPTION:   Global nearshore slope estimations
    INSTITUTION:         Deltares/TU Twente
    PROVIDERS:           {"name": "Deltares/TU Twente", "url": "http://www.de...
    ...                  ...
    DOI:                 https://doi.org/10.5194/essd-11-515-2019
    LONG_NAME:           GLOBAL_DISTRIBUTION_OF_NEARSHORE_SLOPES
    UNITS:               Decimal degrees for Latitude and Longitude, Positive...
    COMMENT:             Error or Warning code from the calculation: 0: No er...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [16]:
# add or change certain variable / coordinate attributes
### dataset attributes is a dictionary of dictionaries
dataset_attributes = {
    "lon": {"standard_name": "longitude", "long_name": "longitude", "units": "degrees_east"},
    "lat": {"standard_name": "latitude", "long_name": "latitude", "units": "degrees_north"},
    "stations": {"long_name": "stations", "units": "1"}, # set to 1 if no unit
    "dc": { "long_name": "Depth of Closure", "units": "m"},
    "slope": { "long_name": "Nearshore Slope", "units": "1"},
    "error_code": { "long_name": "Error", "units": "1"}

}  # specify custom (CF convention) attributes

 # add / overwrite attributes
for k, v in dataset_attributes.items():
    try:
        ds[k].attrs = dataset_attributes[k]
    except:
        continue
ds

<xarray.Dataset>
Dimensions:     (nstations: 783438)
Coordinates:
    lon         (nstations) float64 ...
    lat         (nstations) float64 ...
    error_code  (nstations) int64 ...
Dimensions without coordinates: nstations
Data variables:
    dc          (nstations) float64 ...
    slope       (nstations) float64 ...
Attributes: (12/21)
    TITLE:               Global Distribution of Nearshore Slopes
    TITLE_ABBREVIATION:  gsns
    DESCRIPTION:         Global nearshore slope estimations using an interpol...
    SHORT_DESCRIPTION:   Global nearshore slope estimations
    INSTITUTION:         Deltares/TU Twente
    PROVIDERS:           {"name": "Deltares/TU Twente", "url": "http://www.de...
    ...                  ...
    DOI:                 https://doi.org/10.5194/essd-11-515-2019
    LONG_NAME:           GLOBAL_DISTRIBUTION_OF_NEARSHORE_SLOPES
    UNITS:               Decimal degrees for Latitude and Longitude, Positive...
    COMMENT:             Error or Warning code from the calculation: 0: No er...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [17]:
# Write the xarray dataset to a netCDF file
#Compliant netcdf
dataset_dir_path_CF=str(dataset_dir_path).replace("original","final")


ds.to_netcdf(path=dataset_dir_path_CF)

<IPython.core.display.Javascript object>

### Check CF compliancy modified NetCDF files

In [18]:
# open datasets (only first file, rest is the same)
ds = xr.open_dataset(dataset_dir_path_CF)

# check original dataset
ds

<xarray.Dataset>
Dimensions:     (nstations: 783438)
Coordinates:
    lon         (nstations) float64 ...
    lat         (nstations) float64 ...
    error_code  (nstations) int64 ...
Dimensions without coordinates: nstations
Data variables:
    dc          (nstations) float64 ...
    slope       (nstations) float64 ...
Attributes: (12/21)
    TITLE:               Global Distribution of Nearshore Slopes
    TITLE_ABBREVIATION:  gsns
    DESCRIPTION:         Global nearshore slope estimations using an interpol...
    SHORT_DESCRIPTION:   Global nearshore slope estimations
    INSTITUTION:         Deltares/TU Twente
    PROVIDERS:           {"name": "Deltares/TU Twente", "url": "http://www.de...
    ...                  ...
    DOI:                 https://doi.org/10.5194/essd-11-515-2019
    LONG_NAME:           GLOBAL_DISTRIBUTION_OF_NEARSHORE_SLOPES
    UNITS:               Decimal degrees for Latitude and Longitude, Positive...
    COMMENT:             Error or Warning code from the calculation: 0: No er...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [19]:
%%capture cap --no-stderr
# check original CF compliancy (for first file)

check_compliancy(testfile=dataset_dir_path_CF, 
                 working_dir=CF_dir
                 )

<IPython.core.display.Javascript object>

In [20]:
# save original CF compliancy (for first file)
save_compliancy(cap, testfile=dataset_dir_path_CF, working_dir=CF_dir)

{'P:\\11209197-018-global-coastal-atlas\\MSc_students\\ClenmarRowe\\Data\\All_Datasets\\Orig_Datasets\\02_Exposure\\Slopes-DOC\\Nearshore_Slopes\\Nearshore_Slopes_final.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

In [21]:
# export to zarr in write mode (to overwrite if exists)
ds.to_zarr(str(dataset_dir_path).replace("original","final").replace(".nc", ".zarr"), mode="w")

<IPython.core.display.Javascript object>